In [122]:
import pandas as pd
from github import Github
from github import PaginatedList
import os, json

In [65]:
LABEL_FQ = 'feature-request'
LABEL_BUG = 'bug'
LABEL_OTHER = 'other'

# Construction du dataset

Pour cet exercice, on ne prendra que le titre, le corp et l'ID de l'issue. On va faire 3 classes différentes, 'bug', 'feature-request', 'other'. On considère que chaque input n'a qu'un seul label.

In [90]:
#Fonction qui permet de redéfinir les autres labels que bug et feature-request à other
def filter_label(labels):
    
    if LABEL_FQ  in labels:
        return LABEL_FQ
    elif LABEL_BUG in labels:
        return LABEL_BUG
    
    return LABEL_OTHER

In [149]:
issues = pd.read_csv('./issues.csv') #importation des données téléchargées au préalable
issues = issues.loc[:,['title','body','labels']]

In [143]:
issues

,title,body,labels
0,Panel badge is an odd shape when a single digit,Need to update the css so that this badge beco...,[]
1,custom titlebar : fullscreen very top dragging...,- VSCode Version: Insiders 1.26\r\n- OS Versio...,[]
2,Localized descriptions for built-in extensions...,Fixes #54111,[]
3,editor automatically removing characters from ...,Issue Type: <b>Bug</b>\r\n\r\nthe editor is re...,[]
4,[js] Add auto completion for computed property...,Currently intellisense doesn't work for comput...,"[{'id': 291124272, 'node_id': 'MDU6TGFiZWwyOTE..."
5,Electron 2.0.5,This reverts https://github.com/Microsoft/vsco...,[]
6,"Mac OS X ""Invalid key shortcut terminal"" on ⌃`",I want to open a terminal tab using the shortc...,"[{'id': 362694632, 'node_id': 'MDU6TGFiZWwzNjI..."
7,ES & TS autoimport features enhance,No useful ways to import es & ts modules witho...,"[{'id': 294817667, 'node_id': 'MDU6TGFiZWwyOTQ..."
8,Folder with Chinese path cannot import into wo...,Issue Type: <b>Bug</b>\r\n\r\nPath include Chi...,"[{'id': 362694632, 'node_id': 'MDU6TGFiZWwzNjI..."
9,Cannot uninstall VS Code on Windows Server 201...,Interesting use case here while using Visual S...,"[{'id': 343063018, 'node_id': 'MDU6TGFiZWwzNDM..."


In [164]:
#Transformation des labels. On ne garde qu'un seul label (bug, feature-request et other)
for ind, row in issues.iterrows():
    labels = eval(row['labels'])
    tmp = []

    if len(labels) > 0: # S'il y a au moins un label, 3 possibilitées d'affectation
        for l in labels:
            tmp.append(l['name'])

        new_label = filter_label(tmp)
        issues.loc[ind, 'labels'] = new_label
    else:
        issues.loc[ind, 'labels'] = LABEL_OTHER #Sinon c'est other

In [165]:
issues

,title,body,labels
0,Panel badge is an odd shape when a single digit,Need to update the css so that this badge beco...,other
1,custom titlebar : fullscreen very top dragging...,- VSCode Version: Insiders 1.26\r\n- OS Versio...,other
2,Localized descriptions for built-in extensions...,Fixes #54111,other
3,editor automatically removing characters from ...,Issue Type: <b>Bug</b>\r\n\r\nthe editor is re...,other
4,[js] Add auto completion for computed property...,Currently intellisense doesn't work for comput...,other
5,Electron 2.0.5,This reverts https://github.com/Microsoft/vsco...,other
6,"Mac OS X ""Invalid key shortcut terminal"" on ⌃`",I want to open a terminal tab using the shortc...,other
7,ES & TS autoimport features enhance,No useful ways to import es & ts modules witho...,other
8,Folder with Chinese path cannot import into wo...,Issue Type: <b>Bug</b>\r\n\r\nPath include Chi...,other
9,Cannot uninstall VS Code on Windows Server 201...,Interesting use case here while using Visual S...,other


In [168]:
print(issues.labels.value_counts())
print('Totale : {}'.format(issues.shape[0]))

feature-request    2833
other              1644
bug                 904
Name: labels, dtype: int64
Totale : 5381


On constate un déséquilibre au niveau  du nombre d'éléments par classe. Cela pourra poser des difficultés pour l'apprentissage.

# Préprocessing

Maintenant que nous avons notre dataset, nous allons normaliser nos données, afin d'aider notre classifier à trouver du sens.

In [52]:
for l in labels:
    print(l)

Label(name="*caused-by-extension")
Label(name="*dev-question")
Label(name="*duplicate")
Label(name="*extension-candidate")
Label(name="*not-reproducible")
Label(name="*out-of-scope")
Label(name="*question")
Label(name="External")
Label(name="L10N")
Label(name="VIM")
Label(name="WSL")
Label(name="accessibility")
Label(name="accessibility-jaws")
Label(name="api")
Label(name="api-finalization")
Label(name="api-proposal")
Label(name="as-designed")
Label(name="breadcrumbs")
Label(name="broken-build")
Label(name="bug")
Label(name="candidate")
Label(name="cli")
Label(name="color-palette")
Label(name="config")
Label(name="css-less-scss")
Label(name="debt")
Label(name="debug")
Label(name="debug-console")
Label(name="diff-editor")
Label(name="dropdown")
Label(name="editor")
Label(name="editor-1000-limit")
Label(name="editor-autoclosing")
Label(name="editor-autoindent")
Label(name="editor-brackets")
Label(name="editor-clipboard")
Label(name="editor-code-actions")
Label(name="editor-code-lens")
La

In [53]:
test.title

'Test: debug process termination signal'

In [57]:
issues = rep.get_issues(labels=PaginatedList(['bug','feature-request']))

TypeError: 'module' object is not callable

In [55]:
type(labels)

github.PaginatedList.PaginatedList

In [59]:
PaginatedList.PaginatedList(list_item=['1','2'])

TypeError: __init__() missing 4 required positional arguments: 'contentClass', 'requester', 'firstUrl', and 'firstParams'

In [60]:
test

Issue(title="Test: debug process termination signal", number=54866)

In [61]:
test.number

54866

In [62]:
test.body

'Refs: #54384\r\n\r\n- [ ] os x\r\n- [ ] linux\r\n\r\nComplexity: 3\r\n\r\nVerify:\r\n* Triggering a debug stop command (either from the command palette or the debug toolbar) sends the terminate signal to the debugee (instead of forcefully killing it like before)\r\n* Triggering a debug stop command for the second time forecfully kills the debugee\r\n* Termination signal is not sent for `"attach"` configurations\r\n* Extension host debugging works like before (you can start, reload, restart, quit with no problems)\r\n\r\ntodo @weinand'

In [72]:
labels.totalCount()

AttributeError: 'NoneType' object has no attribute 'startswith'

In [75]:
labels[0]

Label(name="*caused-by-extension")

AttributeError: 'NoneType' object has no attribute 'startswith'

In [99]:
files = os.listdir('./issues/')

In [100]:
files.sort()

In [104]:
dfs = []
for f in files:
    dfs.append(pd.read_json('./issues/' + f))

In [109]:
issues = pd.concat(dfs)

/home/mathieu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [112]:
issues.to_csv('issues.csv')

In [113]:
issues.loc[:,['title','body','labels']]

,title,body,labels
0,Panel badge is an odd shape when a single digit,Need to update the css so that this badge beco...,[]
1,custom titlebar : fullscreen very top dragging...,- VSCode Version: Insiders 1.26\r\n- OS Versio...,[]
2,Localized descriptions for built-in extensions...,Fixes #54111,[]
3,editor automatically removing characters from ...,Issue Type: <b>Bug</b>\r\n\r\nthe editor is re...,[]
4,[js] Add auto completion for computed property...,Currently intellisense doesn't work for comput...,"[{'id': 291124272, 'node_id': 'MDU6TGFiZWwyOTE..."
5,Electron 2.0.5,This reverts https://github.com/Microsoft/vsco...,[]
6,"Mac OS X ""Invalid key shortcut terminal"" on ⌃`",I want to open a terminal tab using the shortc...,"[{'id': 362694632, 'node_id': 'MDU6TGFiZWwzNjI..."
7,ES & TS autoimport features enhance,No useful ways to import es & ts modules witho...,"[{'id': 294817667, 'node_id': 'MDU6TGFiZWwyOTQ..."
8,Folder with Chinese path cannot import into wo...,Issue Type: <b>Bug</b>\r\n\r\nPath include Chi...,"[{'id': 362694632, 'node_id': 'MDU6TGFiZWwzNjI..."
9,Cannot uninstall VS Code on Windows Server 201...,Interesting use case here while using Visual S...,"[{'id': 343063018, 'node_id': 'MDU6TGFiZWwzNDM..."


In [154]:
issues.iloc[4,:].labels

"[{'id': 291124272, 'node_id': 'MDU6TGFiZWwyOTExMjQyNzI=', 'url': 'https://api.github.com/repos/Microsoft/vscode/labels/javascript', 'name': 'javascript', 'color': '1d76db', 'default': False}]"

In [155]:
json.loads(issues.iloc[4,:].labels)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)

In [163]:
type(eval(issues.iloc[0,:].labels))

list

In [131]:
for ind, row in issues.iterrows():
    labels = row['labels']
    tmp = []
    print(labels)
    for l in labels:
        tmp.append(l['name'])
        
    new_label = filter_label(tmp)
    issues.loc[ind, 'labels'] = new_label

test


TypeError: string indices must be integers